In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os 

os.chdir("../")

# sys.path.append("../")

from einops import rearrange
import einops 
import numpy as np
import torch

In [23]:
q = np.random.uniform(size=[32, 8, 128, 10])
k = np.random.uniform(size=[32, 8, 128, 3])

In [29]:
sim = einops.einsum("bhdi, bhdj -> bhij", q, k)

ValueError: The last argument passed to `einops.einsum` must be a string, representing the einsum pattern.

In [27]:
sim2 = np.einsum("bhdi, bhdj -> bhdji", q, k)

In [2]:
from datasets import load_dataset

dataset = load_dataset("fashion_mnist")
IMAGE_SIZE = 28
CHANNELS = 1
BATCH_SIZE = 8

Found cached dataset fashion_mnist (/home/johannes/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/8d6c32399aa01613d96e2cbc9b13638f359ef62bb33612b077b4c247f6ef99c1)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from unet import Unet
from diffusion import Diffusion

unet = Unet(
    scale_channels=32,
    init_channels=128,
    out_channels=1,
    nchannels=CHANNELS,
)

diffusion = Diffusion(
    timesteps=200,
)

In [4]:
from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose(
    [   transforms.Resize(32),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Lambda(lambda t: (t * 2) - 1)
    ]
)

def normalize(examples):
    
    examples["x"] = [transform(image.convert("L")) for image in examples["image"]]
    del examples["image"]

    return examples

transformed_dataset = dataset.with_transform(normalize)

# create dataloader
dataloader = DataLoader(transformed_dataset["train"], batch_size=BATCH_SIZE, shuffle=True)

In [5]:
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

# Writer will output to ./runs/ directory by default
writer = SummaryWriter("runs/test1")
writer.add_scalar("hejsan", 5, 9)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

unet.to(device)

optimizer = torch.optim.Adam(unet.parameters(), lr=1e-3)

iteration = 0

for e in range(100):

    for step, batch in enumerate(dataloader):

        iteration += 1

        x = batch["x"].to(device)

        x_noised, noise, time = diffusion.forward(x)

        noise_predicted = unet.forward(x_noised.to(device), time.to(device))

        loss = diffusion.loss(noise, noise_predicted)

        writer.add_scalar("loss", loss, iteration)

In [8]:
loss

tensor(0.4943, device='cuda:0', grad_fn=<SmoothL1LossBackward0>)

In [17]:
y = unet(batch["x"], time=None)